## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


def f_compute_chisqr(dict_val,dict_sample):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    val_dr=dict_val['hist_val'].copy()
    val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins
    
    sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2
    
    size=len(dict_val['hist_val'])
    l1,l2=int(size*0.3),int(size*0.7)
    keys=['chi_1a','chi_1b','chi_1c','chi_1']
    
    for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
        chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})
    
    idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
#     chisqr_dict.update({'chi_sqr1':})
    
    chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
    chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data
    
    idx=60
    spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
    ### computing the spectral loss chi-square
    chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})
    
    ### computing the spectral loss chi-square
    chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_err'][:idx]**2)})
    
    return chisqr_dict

In [6]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
# s_val=np.load(fname,mmap_mode='r')[10000:20000][:,0,:,:]
s_val=np.load(fname,mmap_mode='r')[:8000][:,0,:,:]

print(s_val.shape)

### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(8000, 128, 128)


In [7]:
df_runs=pd.DataFrame([])

In [8]:
### Load raw images ( a different set of input images for comparison)
dict1={}
dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/train.npy'})
images=f_transform(np.load(dict1['fname'],mmap_mode='r')[:500][:,:,:,0])
dict1['num_img']=images.shape[0]
### Compute spectrum and histograms
dict_sample=f_compute_hist_spect(images,bins)
### Compute chi squares
dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)

dict1.update(dict_sample)
dict1.update(dict_chisqrs)
del(images)
df_runs=df_runs.append(dict1,ignore_index=True)


In [9]:
### Load keras images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
lst=['run5_fixed_cosmology','run6_fixed_cosmology','run7_no_truncated_normal','run8_no_truncated_normal']
prefix='keras_'

for strg,fldr in zip([str(i) for i in range(len(lst))],lst):
    key=prefix+strg
    fname=parent_dir+fldr+'/models/gen_imgs.npy'
    dict1={'label':key,'fname':fname}

    images=np.load(dict1['fname'])[:500]     ### No transform needed here
#     print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


In [10]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201002_064327/'
prefix='pytorch_1_'
lst=[(4,6900),(4,6950),(6,9600),(5,8750),(4,7200),(5,8319),(5,8000)]


for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'*_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)



(128, 128, 128)
0.9976712 -0.9989215
(128, 128, 128)
0.99800277 -0.998225
(128, 128, 128)
0.99605626 -0.99925464
(128, 128, 128)
0.99861085 -0.99770296
pytorch_1_4-7200
(500, 128, 128)
0.99960595 -0.99977577
pytorch_1_5-8319
(500, 128, 128)
0.99949104 -0.9989246
(128, 128, 128)
0.999881 -0.998943


In [66]:


## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201013_151030/'
prefix='pytorch_2_'
lst=[(3,5000),(6,9900),(6,10000),(6,10150),(5,8300)]


for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'*_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)





(128, 128, 128)
0.99916464 -0.99938184
(128, 128, 128)
0.9980439 -0.9937869
(128, 128, 128)
0.9988816 -0.9936874
(128, 128, 128)
0.9978011 -0.99537796
(128, 128, 128)
0.9982418 -0.9965421


In [76]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201013_211359/'
prefix='pytorch_3_'
lst=[(39,62250),(7,12200),(39,62400),(39,61950),(4,7140)]


for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'*_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    print(key)
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
#     print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)




pytorch_3_39-62250
(128, 128, 128)
pytorch_3_7-12200
(128, 128, 128)
pytorch_3_39-62400
(128, 128, 128)
pytorch_3_39-61950
(128, 128, 128)
pytorch_3_4-7140
(500, 128, 128)


In [11]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201015_072548/'
prefix='pytorch_4_'
lst=[(1,2500),(2,3400),(2,3150),(6,10400),(4,7002),(5,8012)]


for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'*_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    print(key)
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
#     print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)





pytorch_4_1-2500
(128, 128, 128)
pytorch_4_2-3400
(128, 128, 128)
pytorch_4_2-3150
(128, 128, 128)
pytorch_4_6-10400
(128, 128, 128)
pytorch_4_4-7002
(500, 128, 128)
pytorch_4_5-8012
(500, 128, 128)


In [ ]:
# # ### Extract a few images generated by Lbann
# parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/generate_images/'
# # fldr='20200806_065947_gen_img_exagan/'
# prefix='lbann_gen_'

# lst=['20200806_070111_gen_img_exagan/']
# names=['1_b64']

# for strg,fldr in zip(names,lst):
#     f_strg=parent_dir+fldr+'dump_outs/trainer0/model0/sgd.testing.epoch.*.step.*_gen_img_instance1_activation_output0.npy'
#     f_list=glob.glob(f_strg)
#     print(f_list)
#     key=prefix+strg
# #     fname=parent_dir+fldr+'/models/gen_imgs.npy'
#     dict1={'label':key,'fname':f_strg}
#     arr=[np.load(fname)[:,0,:,:] for fname in f_list]
#     images=np.vstack(arr)
#     dict1['num_img']=images.shape[0]
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
#     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
#     dict1.update(dict_sample)
#     dict1.update(dict_chisqrs)
#     del(images)
#     df_runs=df_runs.append(dict1,ignore_index=True)

# # print(s_lbann.shape,np.max(s_lbann))


In [ ]:
# ### Extract some LBANN training run data

# parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072639_bsize256_1gpurun/dump_outs/trainer0/model0/'
# lst=[(19, 17461), (19, 17457), (18, 15886), (18, 15887)]
# prefix='lbann_train_1_'
# for count,run in enumerate(lst):
#     epoch,step=run[0],run[1]
#     key=prefix+'{0}-{1}'.format(epoch,step)
#     fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
#     dict1={'label':key,'fname':fname}
#     images=np.load(fname)[:,0,:,:]
#     dict1['num_img']=images.shape[0]
#     print(dict1)
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
#     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
#     dict1.update(dict_sample)
#     dict1.update(dict_chisqrs)
#     del(images)
#     df_runs=df_runs.append(dict1,ignore_index=True)


In [ ]:
# parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072615_bsize256_8gpurun/dump_outs/trainer0/model0/'
# lst=[(19, 17258), (16, 14798), (18, 15860), (17, 15532)]
# prefix='lbann_train_2_'
# for count,run in enumerate(lst):
#     epoch,step=run[0],run[1]
#     key=prefix+'{0}-{1}'.format(epoch,step)
#     fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
#     dict1={'label':key,'fname':fname}
#     images=np.load(fname)[:,0,:,:]
#     dict1['num_img']=images.shape[0]
#     print(dict1)
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
#     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
#     dict1.update(dict_sample)
#     dict1.update(dict_chisqrs)
#     del(images)
#     df_runs=df_runs.append(dict1,ignore_index=True)

In [ ]:

# parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122250_bsize64_8gpurun/dump_outs/trainer0/model0/'
# lst=[(6, 24258), (6, 23967), (9, 32897), (5, 20355),(8,28864)]
# prefix='lbann_train_3_'
# for count,run in enumerate(lst):
#     epoch,step=run[0],run[1]
#     key=prefix+'{0}-{1}'.format(epoch,step)
#     fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
#     dict1={'label':key,'fname':fname}
#     images=np.load(fname)[:,0,:,:]
#     dict1['num_img']=images.shape[0]
#     print(dict1)
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
#     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
#     dict1.update(dict_sample)
#     dict1.update(dict_chisqrs)
#     del(images)
#     df_runs=df_runs.append(dict1,ignore_index=True)

In [ ]:

# parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122131_bsize64_1gpurun/dump_outs/trainer0/model0/'
# lst=[(6, 21560), (5, 18439), (6, 22838), (6, 23443)]
# prefix='lbann_train_4_'
# for count,run in enumerate(lst):
#     epoch,step=run[0],run[1]
#     key=prefix+'{0}-{1}'.format(epoch,step)
#     fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
#     dict1={'label':key,'fname':fname}
#     images=np.load(fname)[:,0,:,:]
#     dict1['num_img']=images.shape[0]
#     print(dict1)
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
#     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
#     dict1.update(dict_sample)
#     dict1.update(dict_chisqrs)
#     del(images)
#     df_runs=df_runs.append(dict1,ignore_index=True)

In [12]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'fname', 'hist_bin_centers', 'hist_err',
       'hist_val', 'label', 'num_img', 'spec_err', 'spec_val'],
      dtype='object')

In [13]:
col_list=['label','fname', 'hist_bin_centers', 'hist_err', 'hist_val','spec_err', 'spec_val','chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2',
       'chi_imgvar', 'chi_spec1', 'chi_spec2', 'num_img']
df_runs=df_runs.reindex(columns=col_list)


In [14]:
df_runs

,label,fname,hist_bin_centers,hist_err,hist_val,spec_err,spec_val,chi_1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_img
0,raw,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0028445197484184732, 0.0020816260811514454,...","[1.121347563774932, 0.9576931243390049, 0.2568...","[230375.97693789843, 1987.871939779571, 1009.5...","[199799556.77036873, 87592.048827034, 62024.13...",0.008450,0.000121,0.000127,0.008202,0.000028,4.089795,0.001933,64.768287,500.0
1,keras_0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.003081765924486072, 0.0023479649643570986, ...","[1.1146491994998429, 0.9643402305198981, 0.260...","[236155.48130319998, 1913.65866089898, 963.081...","[198295986.58007547, 85906.63125591779, 60940....",0.001476,0.000066,0.000471,0.000939,0.000039,4.081442,0.043014,2012.300668,500.0
2,keras_1,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0031456442853499706, 0.002349065715521853, ...","[1.1148820753244182, 0.9669777732485703, 0.261...","[248664.27548104365, 2012.174777339885, 1015.3...","[199125775.3893545, 86439.30616832257, 60426.5...",0.001721,0.000433,0.000196,0.001093,0.000091,4.180616,0.066744,2852.620989,500.0
3,keras_2,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0032299157726108983, 0.00241667307354839, 0...","[1.1439634352627102, 0.9270007224057393, 0.253...","[255385.81330533687, 1877.6099696345757, 1000....","[200294367.72347382, 82519.15650125599, 59058....",0.003428,0.002287,0.000471,0.000670,0.001813,4.139944,0.017024,436.852882,500.0
4,keras_3,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0029680794378720322, 0.002208163391797882, ...","[1.0973298711470636, 0.9833819747814072, 0.269...","[232416.02907518425, 2032.9510454630201, 978.7...","[197739794.9449801, 88968.60526771784, 62091.8...",0.003392,0.001494,0.000660,0.001238,0.001118,4.010530,0.044437,2270.140782,500.0
5,pytorch_1_4-6900,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.005229123704485933, 0.0046812639804679505, ...","[1.1224474677006315, 0.9542432671230308, 0.268...","[373493.2154561559, 3134.997839560339, 1587.63...","[198939945.75595292, 75758.53264936805, 55235....",0.083922,0.003324,0.011814,0.068784,0.000396,7.996223,0.337696,2766.129855,128.0
6,pytorch_1_4-6950,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0053481738250118725, 0.004316593478315989, ...","[1.0951254979568932, 0.9761152413858127, 0.285...","[394203.7936828818, 3618.645949360001, 1621.75...","[197285113.86081547, 77392.35106897354, 54539....",0.061624,0.005916,0.016590,0.039118,0.001870,7.811166,0.270516,2392.392696,128.0
7,pytorch_1_6-9600,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.00828253247118687, 0.006530818089030152, 0....","[1.0257020935314864, 1.0606373240763953, 0.298...","[577999.0591461647, 4187.644999865413, 1950.15...","[193390587.15222764, 106145.79057741165, 66790...",0.184034,0.028944,0.002158,0.152931,0.020805,10.362566,0.262732,1833.085743,128.0
8,pytorch_1_5-8750,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.006991144725811618, 0.005863981474299074, 0...","[1.0392161054548439, 1.0403340683839037, 0.302...","[486182.6148203016, 3506.790873107146, 1668.82...","[194287123.6816132, 83421.01993422862, 58574.0...",0.194877,0.023798,0.006732,0.164347,0.015194,9.420143,0.166826,1590.675360,128.0
9,pytorch_1_4-7200,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002783672366152085, 0.0022684595589339477, ...","[1.1503330728934353, 0.8836046341655227, 0.263...

### Compare chi-squares

In [15]:
chi_keys=['label','chi_1','chi_spec1', 'num_img', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','fname']
df_runs[chi_keys]

,label,chi_1,chi_spec1,num_img,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,fname
0,raw,0.008450,0.001933,500.0,0.000121,0.000127,0.008202,0.000028,4.089795,64.768287,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
1,keras_0,0.001476,0.043014,500.0,0.000066,0.000471,0.000939,0.000039,4.081442,2012.300668,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
2,keras_1,0.001721,0.066744,500.0,0.000433,0.000196,0.001093,0.000091,4.180616,2852.620989,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
3,keras_2,0.003428,0.017024,500.0,0.002287,0.000471,0.000670,0.001813,4.139944,436.852882,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
4,keras_3,0.003392,0.044437,500.0,0.001494,0.000660,0.001238,0.001118,4.010530,2270.140782,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,pytorch_1_4-6900,0.083922,0.337696,128.0,0.003324,0.011814,0.068784,0.000396,7.996223,2766.129855,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,pytorch_1_4-6950,0.061624,0.270516,128.0,0.005916,0.016590,0.039118,0.001870,7.811166,2392.392696,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,pytorch_1_6-9600,0.184034,0.262732,128.0,0.028944,0.002158,0.152931,0.020805,10.362566,1833.085743,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
8,pytorch_1_5-8750,0.194877,0.166826,128.0,0.023798,0.006732,0.164347,0.015194,9.420143,1590.675360,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
9,pytorch_1_4-7200,3.715080,0.582573,500.0,0.018316,0.055134,3.641630,0.008518,6.231101,14621.148897,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...


In [22]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec'],"Invalid mode %s"%(plot_type)

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    
#     plt.xscale('symlog',linthreshx=30)

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')

In [23]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec']))

interactive(children=(SelectMultiple(description='labels_list', options=('raw', 'keras_0', 'keras_1', 'keras_2…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [18]:
col_list=['label','chi_1','chi_spec1','num_img','chi_2','chi_spec2']
df_runs[col_list].head(11)

,label,chi_1,chi_spec1,num_img,chi_2,chi_spec2
0,raw,0.008450,0.001933,500.0,0.000028,64.768287
1,keras_0,0.001476,0.043014,500.0,0.000039,2012.300668
2,keras_1,0.001721,0.066744,500.0,0.000091,2852.620989
3,keras_2,0.003428,0.017024,500.0,0.001813,436.852882
4,keras_3,0.003392,0.044437,500.0,0.001118,2270.140782
5,pytorch_1_4-6900,0.083922,0.337696,128.0,0.000396,2766.129855
6,pytorch_1_4-6950,0.061624,0.270516,128.0,0.001870,2392.392696
7,pytorch_1_6-9600,0.184034,0.262732,128.0,0.020805,1833.085743
8,pytorch_1_5-8750,0.194877,0.166826,128.0,0.015194,1590.675360
9,pytorch_1_4-7200,3.715080,0.582573,500.0,0.008518,14621.148897


In [61]:
f_transform(160)

0.9512195120761451